This is quick iteration to figure out if there's a way to get langsmith tracing to group correctly for map-reduce summarization pipelines

In [1]:
from core import init

# Langchain caching
init()

In [2]:
from core import Seed

import reddit
import reddit.search
import reddit.fetch

target = Seed.init("98point6")

reddit_client = reddit.fetch.init()

reddit_urls = [
    result.link
    for result in reddit.search.find_submissions(
        target, num_results=2
    )
]

reddit_threads = [reddit_client.submission(url=url) for url in reddit_urls]

# only threads with enough comments
reddit_threads = [
    submission for submission in reddit_threads if submission.num_comments >= 2
]

markdowns = [reddit.fetch.submission_to_markdown(thread) for thread in reddit_threads]
markdowns

2024-09-16 15:01:22.616 | DEBUG    | google_search:search:58 - Google search results: {'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - site:reddit.com "98point6""', 'totalResults': '54', 'searchTerms': 'site:reddit.com "98point6""', 'count': 2, 'star

['# Post ID bg7ip2: Internet medicine is awesome, 98point6 was so so helpful for me with +55 score by [(FrugalChef13, Reddit, 2019-04-22)](https://www.reddit.com/r/povertyfinance/comments/bg7ip2/internet_medicine_is_awesome_98point6_was_so_so/)\n**TL;DR- $20 got me an awesome appointment with a nice doctor and a prescription for a medication I could afford that solved my issue.**\n\n*Disclaimer: This particular thing worked well for me so I\'m going to tell you about it. Everyone is different, so it might not work as well (or at all) for you.  Take what you find useful from this post and ignore the rest.  I\'m not compensated or connected to the website I\'m discussing.*\n\nSo like a lot of people on here I\'m usually either uninsured or underinsured.  Right now it\'s underinsured with a high deductible, so when I messed my back up badly enough that I could barely move I freaked.  I\'ve got scoliosis, a fucked up spine, bad knees, and muscles that love to spasm uncontrollably for days 

In [12]:
from langsmith.run_trees import RunTree

# Create a top-level (parent) run
parent_run = RunTree(
    name="Parent Run",
    run_type="chain",
    inputs={"some_input": "value"}
)

parent_run

RunTree(id=UUID('87fd2c3c-3bd6-412a-96bd-54c0b770b8a5'), name='Parent Run', start_time=datetime.datetime(2024, 9, 16, 22, 19, 52, 59662, tzinfo=datetime.timezone.utc), run_type='chain', end_time=None, extra={}, error=None, serialized=None, events=[], inputs={'some_input': 'value'}, outputs={}, reference_example_id=None, parent_run_id=None, tags=[], parent_run=None, child_runs=[], session_name='company-detective-test', session_id=None, dotted_order='20240916T221952059662Z87fd2c3c-3bd6-412a-96bd-54c0b770b8a5', trace_id=UUID('87fd2c3c-3bd6-412a-96bd-54c0b770b8a5'))

In [21]:
from langchain_core.documents import Document
from langchain.chains.summarize import load_summarize_chain
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
import os
from langchain_core.callbacks.manager import trace_as_chain_group
from pprint import pprint

os.environ["LANGCHAIN_PROJECT"] = "company-detective-test"

map_prompt = "Summarize the following Reddit thread. Format the summary as markdown: \n{text}"
map_prompt_template = PromptTemplate(
    template=map_prompt, input_variables=["text"]
)

combine_prompt = "Combine the following summaries into a single markdown document: \n{text}"
combine_prompt_template = PromptTemplate(
    template=combine_prompt, input_variables=["text"]
)

documents = [
    Document(page_content=markdown)
    for markdown in markdowns
]


with trace_as_chain_group("Summarize Company", inputs={"seed": target}) as tracing_callback:

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    summary_chain = load_summarize_chain(
        llm=llm.with_config({"run_name": "Summary mapper", "callbacks": tracing_callback}),
        reduce_llm=llm.with_config({"run_name": "Summary reducer"}),
        chain_type="map_reduce",
        map_prompt=map_prompt_template,
        combine_prompt=combine_prompt_template,
        token_max=30000,
        verbose=False,
        return_intermediate_steps=True,
        # callbacks=tracing_callback,
    )

    result = summary_chain.with_config({"run_name": "Summarize Customer Experience"}).invoke(
        {
            "company": target.company,
            "product": target.product,
            "input_documents": documents,
        },
        # TODO: There's a bug; the langchain config isn't propagated to the map steps so those aren't grouped in the output
        # There's an arg to load_summarize_chain (callbacks) that looks like it should work, but it didn't work when I tried it
        {"callbacks": tracing_callback},
    )

    tracing_callback.on_chain_end({"output": result})

    # parent_run.end(outputs=result)
    # parent_run.post()

    print(result["output_text"])

Error in LangChainTracer.on_chain_start callback: ValidationError(model='Run', errors=[{'loc': ('__root__',), 'msg': "argument of type 'NoneType' is not iterable", 'type': 'type_error'}])
Error in LangChainTracer.on_chain_end callback: TracerException('No indexed run ID f31216c3-86c6-4033-9ab5-bf379aec46fa.')


# Combined Summary of Reddit Threads on 98point6

## Summary of Reddit Thread: Internet Medicine Experience with 98point6

**Post ID:** bg7ip2  
**Author:** FrugalChef13  
**Date:** 2019-04-22  
**[Link to Post](https://www.reddit.com/r/povertyfinance/comments/bg7ip2/internet_medicine_is_awesome_98point6_was_so_so/)**

### TL;DR
The author shares a positive experience using 98point6, an online medical service, which provided an affordable consultation and a prescription for a medication that addressed their health issue.

### Key Points
- The author is underinsured and faced difficulties accessing traditional healthcare due to high costs.
- After experiencing severe back pain, they sought an online solution and found 98point6, which offers a one-year membership for $20 with unlimited online appointments.
- The consultation involved a brief interaction with a chatbot followed by a video call with a doctor who was understanding and supportive.
- The doctor prescribed an anti-inflammatory

In [ ]:
# Error in LangChainTracer.on_chain_start callback: ValidationError(model='Run', errors=[{'loc': ('__root__',), 'msg': "argument of type 'NoneType' is not iterable", 'type': 'type_error'}])
# Parent run 03394b97-e719-4b25-8db6-86870fae49dd not found for run c4184299-a42e-4954-af3d-cc04be460d67. Treating as a root run.
# Parent run 03394b97-e719-4b25-8db6-86870fae49dd not found for run 9461df67-54a7-4836-a8e1-bb1ee8e460f2. Treating as a root run.
# Error in LangChainTracer.on_chain_end callback: TracerException('No indexed run ID 03394b97-e719-4b25-8db6-86870fae49dd.')


In [10]:
tracing_callback

In [25]:
# base_config = {"callbacks": tracing_callback}
base_config = None

In [28]:
{"run_name": "Summary mapper", **(base_config or {})}

{'run_name': 'Summary mapper'}